In [3]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-07-14 14:35:14--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-14 14:35:14 (12.4 MB/s) - ‘minsearch.py.1’ saved [3832/3832]



The above is the code to pull the minsearch.py file from the repository.

### Data Preparation

In [4]:
import minsearch
import json
from openai import OpenAI

In [5]:
client = OpenAI()

In [7]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [8]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [9]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [10]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [11]:
index.fit(documents)

### Functions for RAG (Retrieval Augmented Generation) Model

In [12]:
# Search the documents based on the query and return the results
def search(query):
    boost = {'question': 3.0, 'section': 0.5} # Dictonary to boost the search results based on the field 
    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5 # Number of results to return
    )

    return results

In [13]:
# Build the prompt for the OpenAI API based on the search results
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [14]:
# Get the response from the OpenAI API based on the prompt
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [15]:
# Main function to get the answer based on the query
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [16]:
rag('the course has already started, can I still enroll?')

'Yes, you can still enroll in the course even after it has started. You are eligible to submit the homework, but be aware of the deadlines for the final projects to avoid last-minute submissions.'

### Implementing Elastic Search for Retrieval

Elasticsearch excels at full-text search, allowing you to search and analyze large volumes of text quickly and efficiently. It provides real time indexing of data, making it possible to search and analyze data as soon as it is ingested.

Docker commands to run Elasticsearch:

This commands will create a volume for Elasticsearch data and run the Elasticsearch container with the specified configuration and store the data persistently in the volume.

```bash 
docker volume create elasticsearch_data

docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -v elasticsearch_data:/usr/share/elasticsearch/data \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3


In [17]:
from elasticsearch import Elasticsearch

In [18]:
es_client = Elasticsearch('http://localhost:9200') 

In [19]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": { # Mapping for the fields in the index setting question to be the keyword type and rest to be text type
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions" # Just a name for the index

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [20]:
documents[0]


{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [21]:
from tqdm.auto import tqdm


/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:22<00:00, 42.83it/s]


In [23]:
# Function to search the documents based on the query using ElasticSearch
def elastic_search(query):
    search_query = {
        "size": 5, # Number of results to return 
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"], # Boosting the question field by 3
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp" # Filtering the results based on the course
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query) # Searching the index with the query
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [24]:
# Main function to get the answer based on the query using ElasticSearch
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [25]:
query = 'I just discovered the course. Can I still join it?'

In [26]:
rag(query)

'Yes, you can still join the course even after it has started. You are eligible to submit the homeworks without needing to register. However, please be mindful of deadlines for turning in the final projects and avoid leaving everything for the last minute.'